In [1]:
import pandas as pd
import numpy as np
import requests, csv
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Block group level

In [3]:
#first grab 2021 census data at block group level for CV counties
#set variables
year = 2021
dsource = 'acs'
dname = 'acs5'
#cols = 'NAME,B15003_001E'
cols = 'B01003_001E,B06012_001E,B06012_002E,B19013_001E,B25064_001E,B23025_001E,B23025_002E,B23025_005E,B23025_007E,B23018_001E,B19301_001E,B15003_001E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B17011_001E,B19051_001E,B19051_002E,B19051_003E,B19055_002E,B19055_003E,B19056_001E,B19056_002E,B19057_002E,B19058_002E,B19083_001E,B19113_001E,B01003_001E,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,B19123_002E,B19123_003E'
state = '06'
county = '099,077,047'
block_group = '*'
#list of geographies: https://api.census.gov/data/2022/acs/acs5/geography.html
dcode = '2,12'
keyfile = 'apikey.txt'
outfile = 'nosipopcv2021.txt'

base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'

with open(keyfile) as key:
    api_key = key.read().strip()

data_url = f'{base_url}?get={cols}&for=block%20group:{block_group}&in=state:{state}&in=county:{county}&in=tract:*&key={api_key}'
response = requests.get(data_url)
popdata = response.json()
#for record in popdata:
#    print(record)

with open(outfile, 'w', newline='') as writefile:
    writer = csv.writer(writefile, quoting = csv.QUOTE_ALL, delimiter = ',')
    writer.writerows(popdata)

#readin
tempdf=pd.read_csv('nosipopcv2021.txt', converters={'state': str,'county': str, 'tract': str, 'block_group': str})


In [4]:
#grab missing block_group columns at the tract level
year = 2021
dsource = 'acs'
dname = 'acs5'
#cols = 'NAME,B15003_001E''
cols = 'B06012_001E,B06012_002E,B23018_001E,B19083_001E'
county = '099,077,047'
tract = '*'
#list of geographies: https://api.census.gov/data/2022/acs/acs5/geography.html
dcode = '2,12'
keyfile = 'apikey.txt'
outfile = 'nosimissing-tract2021.txt'

base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'
data_url = f'{base_url}?get={cols}&for=tract:{tract}&in=state:{state}&in=county:{county}&in=tract:*&key={api_key}'
response = requests.get(data_url)
popdata = response.json()
#for record in popdata:
#    print(record)

with open(outfile, 'w', newline='') as writefile:
    writer = csv.writer(writefile, quoting = csv.QUOTE_ALL, delimiter = ',')
    writer.writerows(popdata)

    
#read in missing columns    
missing=pd.read_csv('nosimissing-tract2021.txt', converters={'state': str,'county': str, 'tract': str})



In [5]:
missing.columns

Index(['B06012_001E', 'B06012_002E', 'B23018_001E', 'B19083_001E', 'state',
       'county', 'tract'],
      dtype='object')

In [6]:
#combine 2021 datasets
#make unique geoid for each line in both datasets
tempdf['temp'] = tempdf['state'].astype(str) + tempdf['county'].astype(str) + tempdf['tract'].astype(str)

missing['temp'] = missing['state'].astype(str) + missing['county'].astype(str) + missing['tract'].astype(str)
missing.drop(['state', 'county', 'tract'], axis = 1, inplace = True)

#replace NAs with tract level data
df21 = tempdf.merge(missing, how = 'left', on = 'temp')
df21.drop(['temp', 'B06012_001E_x', 'B06012_002E_x', 'B23018_001E_x', 'B19083_001E_x'], axis = 1, inplace = True)

#rename columns
colnames = ['Population','Median_household_income','Median_gross_rent','Total__Population_16_and_over','Total_In_labor_force','Total_In_labor_force_Civilian_labor_force_Unemployed','Total_Not_in_labor_force','Per_capita_income_in_the_past_12_months_(in_2021_inflation-adjusted_dollars)','Total__Population_25_and_over','Total_Regular_high_school_diploma','Total_GED_or_alternative_credential','Total_Some_college_less_than_1_year','Total_Some_college_1_or_more_years_no_degree','Associates','Bachelors','Masters','Professional','Doctorate','Aggregate_income_deficit_in_the_past_12_months','Total__Households','Total_With_earnings','Total_No_earnings','Total_With_Social_Security_income','Total_No_Social_Security_income','Total__Households','Total_With_Supplemental_Security_Income_(SSI)','Total_With_public_assistance_income','Total_With_cash_public_assistance_or_Food_Stamps/SNAP','Median_family_income_in_the_past_12_months_(in_2021_inflation-adjusted_dollars)','Total_Population','Total__Race','Total_White_alone','Total_Black_or_African_American_alone','Total_American_Indian_and_Alaska_Native_alone','Total_Asian_alone','Total_Native_Hawaiian_and_Other_Pacific_Islander_alone','Total_Some_other_race_alone','Total_Two_or_more_races','Total_With_cash_public_assistance_income_or_households_receiving_Food_Stamps/SNAP_benefits_in_the_past_12_months','Total_No_cash_public_assistance_income_or_household_Food_Stamps/SNAP_benefits_in_the_past_12_months','state', 'county','tract', 'blockgroup','TotalPopforPoverty', 'PopinPoverty','Working_hours','Gini_index']
df21.columns = colnames

In [7]:
#some tracts are missing from 2021 data, try 2022 dataset
year = 2022
dsource = 'acs'
dname = 'acs5'
#cols = 'NAME,B15003_001E'
cols = 'B01003_001E,B06012_001E,B06012_002E,B19013_001E,B25064_001E,B23025_001E,B23025_002E,B23025_005E,B23025_007E,B23018_001E,B19301_001E,B15003_001E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B17011_001E,B19051_001E,B19051_002E,B19051_003E,B19055_002E,B19055_003E,B19056_001E,B19056_002E,B19057_002E,B19058_002E,B19083_001E,B19113_001E,B01003_001E,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,B19123_002E,B19123_003E'
state = '06'
county = '099,077,047'
tract = '*'
block_group = '*'
#list of geographies: https://api.census.gov/data/2022/acs/acs5/geography.html
dcode = '2,12'
keyfile = 'apikey.txt'
outfile = 'nosipopcv2022.txt'

base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'

with open(keyfile) as key:
    api_key = key.read().strip()

data_url = f'{base_url}?get={cols}&for=block%20group:{block_group}&in=state:{state}&in=county:{county}&in=tract:{tract}&key={api_key}'
response = requests.get(data_url)
popdata = response.json()
#for record in popdata:
#    print(record)

with open(outfile, 'w', newline='') as writefile:
    writer = csv.writer(writefile, quoting = csv.QUOTE_ALL, delimiter = ',')
    writer.writerows(popdata)
#readin
tempdf=pd.read_csv('nosipopcv2022.txt', converters={'state': str,'county': str, 'tract': str, 'block_group': str})

In [13]:
#grab unavailable columns for census tract level
year = 2022
dsource = 'acs'
dname = 'acs5'
#cols = 'NAME,B15003_001E''
cols = 'B06012_001E,B06012_002E,B23018_001E,B19083_001E'
county = '099,077,047'
tract = '*'
#list of geographies: https://api.census.gov/data/2022/acs/acs5/geography.html
dcode = '2,12'
keyfile = 'apikey.txt'
outfile = 'nosimissing-tract2022.txt'

base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'
data_url = f'{base_url}?get={cols}&for=tract:{tract}&in=state:{state}&in=county:{county}&in=tract:*&key={api_key}'
response = requests.get(data_url)
popdata = response.json()
#for record in popdata:
#    print(record)

with open(outfile, 'w', newline='') as writefile:
    writer = csv.writer(writefile, quoting = csv.QUOTE_ALL, delimiter = ',')
    writer.writerows(popdata)
#read in missing columns    
missing=pd.read_csv('nosimissing-tract2022.txt', converters={'state': str,'county': str, 'tract': str})

In [14]:
#combine 2022 datasets
#make unique geoid for each line in both datasets
tempdf['temp'] = tempdf['state'].astype(str) + tempdf['county'].astype(str) + tempdf['tract'].astype(str)

missing['temp'] = missing['state'].astype(str) + missing['county'].astype(str) + missing['tract'].astype(str)
missing.drop(['state', 'county', 'tract'], axis = 1, inplace = True)

#replace NAs with tract level data
df22 = tempdf.merge(missing, how = 'left', on = 'temp')
df22.drop(['temp', 'B06012_001E_x', 'B06012_002E_x', 'B23018_001E_x', 'B19083_001E_x'], axis = 1, inplace = True)

#rename columns
colnames = ['Population','Median_household_income','Median_gross_rent','Total__Population_16_and_over','Total_In_labor_force','Total_In_labor_force_Civilian_labor_force_Unemployed','Total_Not_in_labor_force','Per_capita_income_in_the_past_12_months_(in_2021_inflation-adjusted_dollars)','Total__Population_25_and_over','Total_Regular_high_school_diploma','Total_GED_or_alternative_credential','Total_Some_college_less_than_1_year','Total_Some_college_1_or_more_years_no_degree','Associates','Bachelors','Masters','Professional','Doctorate','Aggregate_income_deficit_in_the_past_12_months','Total__Households','Total_With_earnings','Total_No_earnings','Total_With_Social_Security_income','Total_No_Social_Security_income','Total__Households','Total_With_Supplemental_Security_Income_(SSI)','Total_With_public_assistance_income','Total_With_cash_public_assistance_or_Food_Stamps/SNAP','Median_family_income_in_the_past_12_months_(in_2021_inflation-adjusted_dollars)','Total_Population','Total__Race','Total_White_alone','Total_Black_or_African_American_alone','Total_American_Indian_and_Alaska_Native_alone','Total_Asian_alone','Total_Native_Hawaiian_and_Other_Pacific_Islander_alone','Total_Some_other_race_alone','Total_Two_or_more_races','Total_With_cash_public_assistance_income_or_households_receiving_Food_Stamps/SNAP_benefits_in_the_past_12_months','Total_No_cash_public_assistance_income_or_household_Food_Stamps/SNAP_benefits_in_the_past_12_months','state', 'county','tract', 'blockgroup','TotalPopforPoverty', 'PopinPoverty','Working_hours','Gini_index']
df22.columns = colnames

In [15]:
df22['GEOID']= df22['state'].astype(str) + df22['county'].astype(str) + df22['tract'].astype(str) + df22['blockgroup'].astype(str)
df21['GEOID']= df21['state'].astype(str) + df21['county'].astype(str) + df21['tract'].astype(str) + df21['blockgroup'].astype(str)


In [16]:
df = pd.concat([df21, df22])
df = df.drop_duplicates(subset = ['GEOID'])

In [20]:
#make a couple new columns: percent with a degree, place code (equivalent to FIPS), percent households below poverty
df['percent_degree']= (df['Bachelors']+df['Masters']+df['Professional']+df['Doctorate'])/df['Total__Population_25_and_over']

df["Poverty"] = df['PopinPoverty']/df['TotalPopforPoverty']
df['Unemployed'] = df['Total_In_labor_force_Civilian_labor_force_Unemployed']/df['Total_In_labor_force']
df['Food_assistance'] = df['Total_With_cash_public_assistance_income_or_households_receiving_Food_Stamps/SNAP_benefits_in_the_past_12_months']/(df['Total_With_cash_public_assistance_income_or_households_receiving_Food_Stamps/SNAP_benefits_in_the_past_12_months'] + df['Total_No_cash_public_assistance_income_or_household_Food_Stamps/SNAP_benefits_in_the_past_12_months'])


#drop redundant columns 
df.drop(['Bachelors', 'Masters', 'Professional', 'Doctorate', 'PopinPoverty', 'TotalPopforPoverty', 'Total_In_labor_force', 'Total_In_labor_force_Civilian_labor_force_Unemployed', 'Total__Population_16_and_over', 'Total_Not_in_labor_force', 'Total_No_cash_public_assistance_income_or_household_Food_Stamps/SNAP_benefits_in_the_past_12_months', 'Total_With_cash_public_assistance_income_or_households_receiving_Food_Stamps/SNAP_benefits_in_the_past_12_months'], axis = 1, inplace = True)

df.GEOID = df.GEOID.str.zfill(12)

df.replace(-666666666, np.nan, inplace=True)
df.replace(-999999999, np.nan, inplace=True)
df.replace(-888888888, np.nan, inplace=True)
df.to_csv('nosi2021census.csv')




## Tract level

In [2]:
#tract level 2017
year = 2017
dsource = 'acs'
dname = 'acs5'
cols = 'B01003_001E,B19013_001E,B25031_001E,B27011_001E,B27011_002E,B28007_015E,B28007_009E,B19301_001E,B15003_001E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B17008_001E,B19051_001E,B19051_002E,B19051_003E,B19055_002E,B19055_003E,B19056_001E,B19056_002E,B19057_002E,B19058_002E,B19113_001E,B02001_001E,B00001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,B07012_001E,B07012_002E,B23018_001E,B19083_001E,B19058_003E'
county = '*'
state = '06'
tract = '*'
#list of geographies: https://api.census.gov/data/2022/acs/acs5/geography.html
#2015 acs1 subject tables variables
#https://api.census.gov/data/2015/acs/acs1/subject/variables
dcode = '2,12'
keyfile = 'apikey.txt'
outfile = 'nosi-tract2017.txt'


base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'

with open(keyfile) as key:
    api_key = key.read().strip()
    
data_url = f'{base_url}?get={cols}&for=tract:{tract}&in=state:{state}&in=county:{county}&in=tract:*&key={api_key}'
response = requests.get(data_url)
popdata = response.json()

with open(outfile, 'w', newline='') as writefile:
    writer = csv.writer(writefile, quoting = csv.QUOTE_ALL, delimiter = ',')
    writer.writerows(popdata)

    
#readin   
tract17=pd.read_csv('nosi-tract2017.txt', converters={'state': str,'county': str, 'tract': str})


In [3]:
tract17.columns

Index(['B01003_001E', 'B19013_001E', 'B25031_001E', 'B27011_001E',
       'B27011_002E', 'B28007_015E', 'B28007_009E', 'B19301_001E',
       'B15003_001E', 'B15003_017E', 'B15003_018E', 'B15003_019E',
       'B15003_020E', 'B15003_021E', 'B15003_022E', 'B15003_023E',
       'B15003_024E', 'B15003_025E', 'B17008_001E', 'B19051_001E',
       'B19051_002E', 'B19051_003E', 'B19055_002E', 'B19055_003E',
       'B19056_001E', 'B19056_002E', 'B19057_002E', 'B19058_002E',
       'B19113_001E', 'B02001_001E', 'B00001_001E', 'B02001_002E',
       'B02001_003E', 'B02001_004E', 'B02001_005E', 'B02001_006E',
       'B02001_007E', 'B02001_008E', 'B07012_001E', 'B07012_002E',
       'B23018_001E', 'B19083_001E', 'B19058_003E', 'state', 'county',
       'tract'],
      dtype='object')

In [4]:
#rename columns
colnames = ['Population','Median_household_income','Median_gross_rent','Total__Population_16_and_over','Total_In_labor_force','Total_Not_in_labor_force','Total_In_labor_force_Civilian_labor_force_Unemployed','Per_capita_income_in_the_past_12_months_(in_2021_inflation-adjusted_dollars)','Total__Population_25_and_over','Total_Regular_high_school_diploma','Total_GED_or_alternative_credential','Total_Some_college_less_than_1_year','Total_Some_college_1_or_more_years_no_degree','Associates','Bachelors','Masters','Professional','Doctorate','Aggregate_income_deficit_in_the_past_12_months','Total__Households','Total_With_earnings','Total_No_earnings','Total_With_Social_Security_income','Total_No_Social_Security_income','Total__Households','Total_With_Supplemental_Security_Income_(SSI)','Total_With_public_assistance_income','Total_With_cash_public_assistance_income_or_households_receiving_Food_Stamps/SNAP_benefits_in_the_past_12_months','Median_family_income_in_the_past_12_months_(in_2021_inflation-adjusted_dollars)','Total__Race','Total_Population','Total_White_alone','Total_Black_or_African_American_alone','Total_American_Indian_and_Alaska_Native_alone','Total_Asian_alone','Total_Native_Hawaiian_and_Other_Pacific_Islander_alone','Total_Some_other_race_alone','Total_Two_or_more_races','TotalPopforPoverty','PopinPoverty','Working_hours','Gini_index','Total_No_cash_public_assistance_income_or_household_Food_Stamps/SNAP_benefits_in_the_past_12_months','state','county','tract']
tract17.columns = colnames
tract17['GEOID']= tract17['state'].astype(str) + tract17['county'].astype(str) + tract17['tract'].astype(str)


#make a couple new columns: percent with a degree, place code (equivalent to FIPS), percent households below poverty
tract17['percent_degree']= (tract17['Bachelors']+tract17['Masters']+tract17['Professional']+tract17['Doctorate'])/tract17['Total__Population_25_and_over']

tract17["Poverty"] = tract17['PopinPoverty']/tract17['TotalPopforPoverty']
tract17['Unemployed'] = tract17['Total_In_labor_force_Civilian_labor_force_Unemployed']/tract17['Total_In_labor_force']




In [5]:
tract17['assistance'] = tract17['Total_With_public_assistance_income']/(tract17['Total_With_cash_public_assistance_income_or_households_receiving_Food_Stamps/SNAP_benefits_in_the_past_12_months'] + tract17['Total_No_cash_public_assistance_income_or_household_Food_Stamps/SNAP_benefits_in_the_past_12_months'])


#drop redundant columns 
tract17.drop(['Bachelors', 'Masters', 'Professional', 'Doctorate', 'PopinPoverty', 'TotalPopforPoverty', 'Total_In_labor_force', 'Total_In_labor_force_Civilian_labor_force_Unemployed', 'Total__Population_16_and_over', 'Total_Not_in_labor_force', 'Total_No_cash_public_assistance_income_or_household_Food_Stamps/SNAP_benefits_in_the_past_12_months', 'Total_With_cash_public_assistance_income_or_households_receiving_Food_Stamps/SNAP_benefits_in_the_past_12_months'], axis = 1, inplace = True)

tract17.GEOID = tract17.GEOID.str.zfill(11)

tract17.replace(-666666666, np.nan, inplace=True)
tract17.replace(-999999999, np.nan, inplace=True)
tract17.replace(-888888888, np.nan, inplace=True)
tract17.to_csv('nosi2017census.csv')

In [6]:
tract17.columns

Index(['Population', 'Median_household_income', 'Median_gross_rent',
       'Per_capita_income_in_the_past_12_months_(in_2021_inflation-adjusted_dollars)',
       'Total__Population_25_and_over', 'Total_Regular_high_school_diploma',
       'Total_GED_or_alternative_credential',
       'Total_Some_college_less_than_1_year',
       'Total_Some_college_1_or_more_years_no_degree', 'Associates',
       'Aggregate_income_deficit_in_the_past_12_months', 'Total__Households',
       'Total_With_earnings', 'Total_No_earnings',
       'Total_With_Social_Security_income', 'Total_No_Social_Security_income',
       'Total__Households', 'Total_With_Supplemental_Security_Income_(SSI)',
       'Total_With_public_assistance_income',
       'Median_family_income_in_the_past_12_months_(in_2021_inflation-adjusted_dollars)',
       'Total__Race', 'Total_Population', 'Total_White_alone',
       'Total_Black_or_African_American_alone',
       'Total_American_Indian_and_Alaska_Native_alone', 'Total_Asian_alone